# Fiddling around with making Bungie API calls to get json data 

In [66]:
import pandas as pd
import requests
import os

In [107]:
class BungieData(object):

    def __init__(self, api_key, session=None):
        '''
        session: an object from login.py defining a user session by 
        logging into Bungie.net (oAuth verified)
        '''
        self.session = session
        self.api_key = api_key

    def get_userdata(self, user_id):
        '''
        Use old Destiny endpoint
        '''
        site_call = "https://bungie.net/Platform/User/GetBungieNetUserById/" + str(user_id)
        request = requests.get(site_call, 
                                headers={"X-API-Key":self.api_key})
        return request.json()
    
    def get_userdata_v2(self, user_id):
        '''
        Use new Destiny 2 endpoint for PSN player 
        '''
        site_call = "https://bungie.net/Platform/Destiny2/2/Profile/" + str(user_id)
        request = requests.get(site_call, 
                                headers={"X-API-Key":self.api_key})
        return request.json()
    
    def get_postGameStats(self, game_id):
        site_call = "https://bungie.net/Platform/Destiny2/Stats/PostGameCarnageReport/" + str(game_id)
        request = requests.get(site_call, 
                                headers={"X-API-Key":self.api_key})
        return request.json()
    
    def get_playerByTagName(self, gamertag):
        site_call = "https://bungie.net/Platform/Destiny2/SearchDestinyPlayer/2/" + gamertag
        request = requests.get(site_call, 
                                headers={"X-API-Key":self.api_key})
        return request.json()


In [105]:
bungie = BungieData(api_key=os.environ["BUNGIE_API_KEY"]) # Never put your keys in code... export 'em!

In [106]:
# Get my own Bungie MembershipID
player_name = bungie.get_playerByTagName("BalancedSeeker6")
player_name

{u'ErrorCode': 1,
 u'ErrorStatus': u'Success',
 u'Message': u'Ok',
 u'MessageData': {},
 u'Response': [{u'displayName': u'BalancedSeeker6',
   u'iconPath': u'/img/theme/destiny/icons/icon_psn.png',
   u'membershipId': u'4611686018453385415',
   u'membershipType': 2}],
 u'ThrottleSeconds': 0}

In [108]:
# Get my public user info by ID which isn't working for my ID for some reason yet to be determined
content1 = bungie.get_userdata(4611686018453385415)
content1 

{u'ErrorCode': 205,
 u'ErrorStatus': u'UserCannotFindRequestedUser',
 u'Message': u'The requested user could not be found.',
 u'MessageData': {},
 u'ThrottleSeconds': 0}

In [109]:
# Using the new Destiny 2 endpoint is throwing an error for now too
content2 = bungie.get_userdata_v2(4611686018453385415)
content2

{u'ErrorCode': 18,
 u'ErrorStatus': u'InvalidParameters',
 u'Message': u'The input parameters were invalid, please enter valid input, and try again.',
 u'MessageData': {},
 u'ThrottleSeconds': 0}

In [110]:
# Get a single game's stats
game_stats = bungie.get_postGameStats(100)
game_stats

{u'ErrorCode': 1,
 u'ErrorStatus': u'Success',
 u'Message': u'Ok',
 u'MessageData': {},
 u'Response': {u'activityDetails': {u'directorActivityHash': 1658347443,
   u'instanceId': u'100',
   u'isPrivate': False,
   u'mode': 2,
   u'modes': [7, 2],
   u'referenceId': 1658347443},
  u'entries': [{u'characterId': u'2305843009260533952',
    u'extended': {u'values': {u'precisionKills': {u'basic': {u'displayValue': u'55',
        u'value': 55.0}}},
     u'weapons': [{u'referenceId': 53159280,
       u'values': {u'uniqueWeaponKills': {u'basic': {u'displayValue': u'4',
          u'value': 4.0}},
        u'uniqueWeaponKillsPrecisionKills': {u'basic': {u'displayValue': u'75%',
          u'value': 0.75}},
        u'uniqueWeaponPrecisionKills': {u'basic': {u'displayValue': u'3',
          u'value': 3.0}}}},
      {u'referenceId': 4024037919,
       u'values': {u'uniqueWeaponKills': {u'basic': {u'displayValue': u'75',
          u'value': 75.0}},
        u'uniqueWeaponKillsPrecisionKills': {u'basic'